In [ ]:
%%writefile .env
#OPENAI_API_KEY="sk-proj-"
#TAVILY_API_KEY=tvly-dev-

Writing .env


In [ ]:
pip install -q openai-agents==0.2.2 python-dotenv requests pydantic

In [4]:
import os
import requests
from IPython.display import display, Markdown
from openai import OpenAI
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool, SQLiteSession
from typing_extensions import TypedDict
from pydantic import BaseModel

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

print("✅ API keys loaded")
print(f"  OpenAI … {openai_api_key[:5]}***")
print(f"  Tavily … {tavily_api_key[:5]}***")

✅ API keys loaded
  OpenAI … sk-pr***
  Tavily … tvly-***


In [5]:
def print_markdown(text):
    display(Markdown(text))

In [6]:
class TavilySearchParams(TypedDict):
    query: str
    max_results: int

In [7]:
@function_tool
def tavily_search(params: TavilySearchParams) -> str:
    url = "https://api.tavily.com/search"
    headers = {"Content-Type": "application/json"}
    payload = {
        "api_key": tavily_api_key,
        "query": params["query"],
        "max_results": params.get("max_results", 3),
    }
    response = requests.post(url, json = payload, headers = headers)
    if response.status_code == 200:
        results = response.json().get("results", [])
        summary = "\n".join([f"- {r['title']}: {r['content']}" for r in results])
        return summary if summary else "No relevant results found."
    else:
        return f"Tavily API error: {response.status_code}"

print("✅ Tavily search tool ready.")

✅ Tavily search tool ready.


In [8]:
# Let's define our Researcher AI Agent

# Define a Pydantic model for the agent's output.
# This ensures the agent's final output will have a single field: "summary" (string).
# Note that this will be used in both the researcher and analyst agents
class AnalysisSummary(BaseModel):
    summary: str


# Create an AI agent called "Researcher"
researcher_agent = Agent(name = "Researcher",
                         instructions = """
## Context
You are a research agent with access to the Tavily search tool.

## Instruction
Given a user query, use the Tavily search tool to find relevant information and summarize the key findings.

## Input
- A research query from the user.

## Output
- A summary of key findings in a maximum of 5 bullet points.
""",
    model = "gpt-4.1-mini",
    tools = [tavily_search],
    output_type = AnalysisSummary)

print("✅ Researcher AI agent is now ready")

✅ Researcher AI agent is now ready


In [ ]:
researcher_result = await Runner.run(researcher_agent, "Why is Labubu so popular and what are the rarest Labubu collectibles?")
researcher_result

In [9]:
print_markdown(f"### 🤖 Agent’s Answer\n{researcher_result.final_output.summary}")

### 🤖 Agent’s Answer
- Labubu gained popularity largely due to its creation by illustrator Kasing Lung and its integration into POP MART's blind box format, which adds an element of surprise and collectability.
- The Labubu 'Have a Seat' series is a recent popular release with limited availability, making it highly sought after by collectors.
- Blind box collectors are particularly drawn to the "secret" chase figures within the Labubu series, enhancing the thrill of collection.
- Among the rarest Labubu collectibles is the "Pink Microwave" hidden version from the Labubu Convenience Store blind box series, known for its extremely low pull rate of 1 in several thousand.
- Labubu is part of a broader trend of popular collectible dolls and art toys, often ranking in top lists on platforms like TikTok and being featured in various collectible series by POP MART.

In [9]:
analyst_agent = Agent(name = "Analyst",
                      instructions = """
## Context
You are an analyst who receives research notes generated by the research agent.

## Instruction
Given the research notes, analyze the content and extract key trends, risks, or insights.

## Input
- Research notes (summaries of findings from the research agent).

## Output
- A concise analysis (no more than 2 paragraphs) highlighting key trends, risks, or insights.
""",
    model = "gpt-4.1-mini",
    output_type = AnalysisSummary,
)

print("✅ Analyst AI agent us now ready")

✅ Analyst AI agent us now ready


In [ ]:
researcher_result = await Runner.run(researcher_agent, "Electric vehicles batteries trend in 2025")
researcher_result

In [ ]:
analyst_result = await Runner.run(analyst_agent, researcher_result.final_output.summary)
researcher_result

In [13]:
print_markdown(f"### 🤖 Agent’s Answer\n{analyst_result.final_output.summary}")

### 🤖 Agent’s Answer
The EV battery market in 2025 is characterized by both consolidation and geographic diversification, driven by government initiatives to secure supply chains. Key technology trends include the expansion of mid-nickel NMC and L(M)FP batteries, increased use of silicon anodes, and pioneering next-generation battery technologies that underpin EV growth. Additionally, integration of vehicle-to-grid systems is enhancing driving range while enabling energy resale, which supports grid stability.

Regulatory shifts, such as heightened taxes on internal combustion and hybrid vehicles starting April 2025, are expected to accelerate the shift towards battery electric vehicles. This market momentum is reflected in a projected 25% growth in global EV sales and a near doubling of lithium-ion battery manufacturing capacity to 3.8 TWh by the end of 2025, highlighting strong industry expansion but also potential supply chain and sustainability challenges.

In [10]:
class FinalReport(BaseModel):
    short_summary: str  # A brief 2–3 sentence executive summary
    markdown_report: str    # A detailed report in Markdown format (at least 500 words)
    follow_up_questions: list[str]    # A list of 3–5 suggested follow-up research questions


# Create an AI agent called "Writer"
writer_agent = Agent(
    name = "Writer",
    instructions = """
You are a senior market analyst tasked with generating an executive-level research report.

## Context
You will receive:
- The original user query (the research question or topic of interest).
- Summaries and analyses generated by helper tools: 'researcher_agent' (for bullet-point research findings) and 'analyst_agent' (for key trends, risks, or insights).

You may call these helper tools as needed to gather additional information or clarify findings.

## Instructions
Your job is to synthesize all available information and produce the following outputs:
1. **Executive Summary:** Write a concise summary (2–3 sentences) highlighting the most important findings relevant to the original query.
2. **Detailed Markdown Report:** Compose a comprehensive, well-structured report in markdown format (at least 500 words) that covers key findings, context, implications, and supporting evidence.
3. **Follow-up Research Questions:** Suggest 3–5 thoughtful follow-up questions for further investigation.

## Input
- The original user query.
- Summaries and analyses from the helper tools.

## Output
Return a structured object with:
- `short_summary`: The executive summary.
- `markdown_report`: The detailed markdown report.
- `follow_up_questions`: A list of 3–5 follow-up research questions.
""",
    model = "gpt-4.1-mini",
    output_type = FinalReport,
)

print("✅ Writer AI Agent is now ready")

✅ Writer AI Agent is now ready


In [11]:
# Create a session to store and share information between agents
session = SQLiteSession("research_agent_practice")

# Define an async function that takes the user's question and runs the agents
async def manager_run(user_query: str):
    # Show the user’s question in a nice markdown format
    print_markdown(f"**User's Request:** {user_query}")

    # Step 1: Run the researcher agent to gather information about the user's query
    researcher_result = await Runner.run(researcher_agent, user_query, session = session)
    research_summary = researcher_result.final_output.summary  # Get the short research summary

    # Step 2: Run the analyst agent to analyze the research findings
    analyst_result = await Runner.run(analyst_agent, research_summary, session = session)
    analysis_summary = analyst_result.final_output.summary  # Get the short analysis summary

    # Step 3: Prepare the combined input for the writer agent
    input_for_writer = (
        f"Original query: {user_query}\n"
        f"Research summary: {research_summary}\n"
        f"Analysis summary: {analysis_summary}"
    )

    # Step 4: Run the writer agent to create the final report
    writer_result = await Runner.run(writer_agent, input_for_writer, session = session)
    final_output: FinalReport = writer_result.final_output  # The final report from the writer

    # Step 5: Display the results in a nice, clear format
    print_markdown("---")
    print_markdown(f"### 📝 **Short Summary:**\n{final_output.short_summary}")  # Quick overview
    print_markdown("\n\n-----------------\n\n")
    print_markdown(f"### 📄 **Full Report (Markdown):**\n{final_output.markdown_report}")  # Detailed report
    print_markdown("\n\n-----------------\n\n")
    print_markdown(
        "### 🔍 **Follow-Up Questions:**\n- "
        + "\n- ".join(final_output.follow_up_questions)  # Any extra questions the user might explore
    )
    print_markdown("\n\n-----------------\n\n")

In [12]:
await manager_run("How new H!B Visa rule going to impact students on OPT right now in terms of getting jobs and various other aspects?")

**User's Request:** How new H!B Visa rule going to impact students on OPT right now in terms of getting jobs and various other aspects?

---

### 📝 **Short Summary:**
The new H-1B visa rules and proposed legislative changes significantly tighten the pathway for OPT students transitioning to H-1B employment, especially impacting Indian students who are the largest group of OPT holders. Although the Cap-Gap extension offers temporary relief, stricter wage requirements, potential elimination of OPT, and increased petition costs pose considerable challenges for securing jobs and maintaining legal work status in the U.S.



-----------------



### 📄 **Full Report (Markdown):**
# Impact of New H-1B Visa Rules on Students on OPT

## Executive Overview
Recent changes and proposed legislation around the H-1B visa system have ushered in a more stringent regulatory environment for international students currently on Optional Practical Training (OPT). This is particularly consequential for Indian students, who make up the largest segment of F-1 OPT visa holders in the United States. These adjustments affect job acquisition, the ability to maintain work authorization, and overall career planning strategies.

## Key Findings

### Increased Difficulty in Foreign Hiring
The new H-1B visa rules are aligned with efforts to tighten immigration pathways, making it harder for U.S. employers to hire international graduates. The impact is especially pronounced for Indian nationals due to their significant representation among OPT students.

### Importance and Limitations of the Cap-Gap Extension
OPT students who have pending or approved H-1B petitions filed before their OPT or STEM OPT period ends may leverage the Cap-Gap extension to continue working legally while awaiting H-1B status activation. However, this extension is contingent on the approval and maintenance of the H-1B petition. If a petition is denied, revoked, or withdrawn, the Cap-Gap extension immediately terminates, and the student enters a 60-day grace period with limited options.

### Risk of Denial for Late Applications
There is an elevated risk associated with STEM OPT extension applications submitted after the expiration of the OPT EAD but during the Cap-Gap period. Such applications have been found to carry a high probability of denial, increasing uncertainty for students relying on these extensions.

### Proposed Legislative Changes and Financial Challenges
Proposals currently under consideration aim to raise the H-1B visa wage floor significantly, from approximately $60,000 to $150,000. Additionally, there are discussions about ending the OPT program itself, which would severely constrain the ability of international students to gain post-graduation work experience.

Starting in 2025, a new $100,000 fee requirement will apply to many H-1B specialty occupation petitions, further increasing the cost and complexity of obtaining these visas.

## Implications for OPT Students

- **Job Market Access:** OPT students may find fewer opportunities due to employers' increased hesitance given the higher wage requirements and additional visa fees.

- **Work Authorization Stability:** Reliance on the Cap-Gap extension is precarious; any negative outcome in H-1B petitions risks abrupt loss of work status.

- **Financial and Administrative Burdens:** Higher fees and wage floors may discourage smaller employers from sponsoring H-1B visas, reducing options for OPT students.

- **Long-Term Career Planning:** The potential elimination of OPT could force students to seek alternative pathways internationally or modify career expectations.

## Conclusion
The evolving H-1B visa landscape poses significant challenges for students on OPT, particularly those from India. Access to U.S. employment, continued work authorization, and financial feasibility of visa petitions all face increased hurdles. Students and stakeholders must stay informed and consider contingency plans amid these regulatory shifts.

---

### References
- USCIS guidelines on Cap-Gap extensions
- Legislative proposals to raise H-1B wage floors
- Announced fee updates for H-1B visa petitions
- Analysis from immigration policy experts and education consultants



-----------------



### 🔍 **Follow-Up Questions:**
- How will the increased H-1B wage floor specifically affect employers in tech versus other industries employing OPT students?
- What alternative visa options exist for OPT students if the H-1B process becomes prohibitively difficult or costly?
- How can universities and international student offices better prepare students for these changes?
- What are the projected long-term impacts on the U.S. labor market if OPT and H-1B reforms reduce international student employment?
- How effective are Cap-Gap extensions in practice, and what legal reforms could improve protections for OPT students?



-----------------

